In [ ]:
from dotenv import load_dotenv

load_dotenv()

import os

project_name = "wanted_2nd_langchain_outputparser_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(temperature=0.1, model_name="gpt-4.1-mini", verbose=True)

In [3]:
from pydantic import BaseModel, Field, ValidationError
from langchain_core.output_parsers import PydanticOutputParser


class ReviewSummary(BaseModel):
    title: str
    bullets: list[str] = Field(description="주요 리뷰 사항", min_items=3, max_items=5)
    tone: str = Field(description="리뷰에 대한 평가")


parser = PydanticOutputParser(pydantic_object=ReviewSummary)
parser

PydanticOutputParser(pydantic_object=<class '__main__.ReviewSummary'>)

In [4]:
fmt = parser.get_format_instructions()
fmt

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"title": "Title", "type": "string"}, "bullets": {"description": "주요 리뷰 사항", "items": {"type": "string"}, "maxItems": 5, "minItems": 3, "title": "Bullets", "type": "array"}, "tone": {"description": "리뷰에 대한 평가", "title": "Tone", "type": "string"}}, "required": ["title", "bullets", "tone"]}\n```'

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "JSON으로만 출력. \n\n양식: {fmt}"),
        ("user", "다음 텍스트를 의사 결정용으로 요약:\n\n{text}"),
    ]
).partial(fmt=fmt)

In [6]:
prompt

ChatPromptTemplate(input_variables=['text'], input_types={}, partial_variables={'fmt': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"title": "Title", "type": "string"}, "bullets": {"description": "주요 리뷰 사항", "items": {"type": "string"}, "maxItems": 5, "minItems": 3, "title": "Bullets", "type": "array"}, "tone": {"description": "리뷰에 대한 평가", "title": "Tone", "type": "string"}}, "required": ["title", "bullets", "tone"]}\n```'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['fmt'], input_types={}, partial_variables={}, template

In [8]:
chain = prompt | model | parser
text = """
버튼 하나로 부드럽고 쫀쫀한 거품

디자인&컬러 깔끔하고 간편한 조작으로 사용하기는 쉽네요.
거품기 부품 스프링까지 쉽게 분리되어 세척이 간편하고 위생적이다.
우유는 눈금선까지 넣을 경우 100ml 사용된다.

사실 밀크프로더 사용 경험이 없어서 많이 설레였는데,
정말 단순.
중앙 버튼을 1초 누르면 따뜻한 거품,
3초 누르면 차가운 거품을 만들 수 있어요.

거품 만들기 시작을 누른 후에 자동 정지는 없고 적당한 선에서 다시 시작 버튼을 눌러줘야 되네요.

설명서가 초간단이라서 조금 아쉽네요.
시작 누르고 몇 초 후 정지를 눌러야 할지 반복적인 사용으로 터득해야 할 것 같네요.
거품은 쫀쫀하고 실키해요~bb

라떼고 자동 머신에서 카푸치노 만들어 마시다가
직접 만들었더니 조금 번거롭지만
요 이쁜놈 장식품되지 않게 자주 사용해보고
좋은 TIP 터득하면 한 달 후기 써볼까?
생각 중...

타브랜드에 비해 가격은 좀 비싸지만
성능이 중요하지!
디자인도 이쁨!

별 한 개 뺀 것은,
사용방법 설명이 자세하지 않고,
택배 걸 포장이 비닐이라서 안전 배송에 염려 됨.

가성비 성능에 비해 비싸요
성능 기대 이상 뛰어났어요
조작 편리성 조작이 직관적이에요
"""

In [ ]:
result = chain.invoke({"text": text})

In [10]:
result.title

'간편한 조작과 뛰어난 거품 품질의 밀크프로더 리뷰'

In [11]:
print(type(result))

<class '__main__.ReviewSummary'>


In [12]:
result.model_dump()

{'title': '간편한 조작과 뛰어난 거품 품질의 밀크프로더 리뷰',
 'bullets': ['버튼 하나로 부드럽고 쫀쫀한 거품을 쉽게 만들 수 있음',
  '깔끔한 디자인과 간편한 조작, 부품 분리가 쉬워 세척과 위생 관리가 편리함',
  '자동 정지 기능이 없어 사용법 숙지가 필요하며 설명서가 다소 부족함',
  '가격은 다소 높지만 성능과 디자인 면에서 만족스러움',
  '포장 방식이 비닐로 되어 있어 배송 안전성에 대한 우려가 있음'],
 'tone': '긍정적'}

In [13]:
try:
    result = chain.invoke({"text": text})
    print(result)
except ValidationError as e:
    print("ValidationError:", e)

title='간편한 조작과 뛰어난 거품 품질의 밀크프로더 리뷰' bullets=['버튼 하나로 부드럽고 쫀쫀한 거품을 쉽게 만들 수 있음', '깔끔한 디자인과 간편한 조작, 부품 분리가 쉬워 세척과 위생 관리가 용이함', '자동 정지 기능이 없어 사용법 숙지가 필요하며 설명서가 다소 부족함', '가격은 다소 높지만 성능과 디자인 면에서 만족스러움', '포장 방식이 비닐로 되어 있어 배송 안전성에 대한 우려가 있음'] tone='긍정적'
